# Objectives

- To impute cost for Rx encounters or Certified Pharmacy Claims
- Original Work:
https://app.snowflake.com/us-west-2/komodohealth/w2H1m36fPpSq#query


In [1]:
## Import required libraries 

import snowflake.connector
import getpass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)
import plotly.graph_objects as go


In [2]:
import sys
sys.path.append('/home/jovyan/credentials')
import jupytertoolz_credentials as jt

In [4]:
connection = snowflake.connector.connect(user='YWEI', password=getpass.getpass(), account='komodohealth')
connection.cursor().execute('USE ROLE ' + ROLE)
connection.cursor().execute('USE WAREHOUSE ' + WAREHOUSE)

def read_sql(sql, connection = connection):
    df = pd.read_sql(sql, connection)
    return df

def execute_sql(sql, connection = connection):
    #pd.io.sql.execute(sql, connection)
    connection.cursor().execute(sql)
    
execute_sql(f"USE ROLE ANALYST")
execute_sql(f"USE database {DATABASE}")
execute_sql(f"USE SCHEMA {SCHEMA}")

········


### Imputation Logic
https://app.diagrams.net/#G1hkkMMHyEX6y0DgAX3yHb4g3CGMDS-v22

Final hierarchy:
1. Cleaned Allowed Amount
2. Quantity_Final * Unit_Price_Final
    1. Cleaned Quantity
    2. NDC/Cleaned Days Supply -> Imputed Quantity
    3. NDC -> Imputed Quantity
    4. Median Unit Price
    5. NADAC Unit Price 
    6. ASP Unit Price
3. NDC -> Imputed Price

In [17]:
## Snowflake tables 
ROLE = 'ANALYST'
WAREHOUSE = 'XLARGE_WH'
DATABASE = 'SANDBOX_KOMODO'  
SCHEMA = 'PROJECT_CURRENCY'

## Table reference
rx_version = '20220511'
rx_enc = f"MAP_ENCOUNTERS.RX_ENCOUNTERS_{rx_version}.RX_ENCOUNTER_LS_GA"
rx_claim = f"MAP_CERTIFIED_CLAIMS.PUBLIC.VIEW_CERTIFIED_CLAIMS_PHARMACY_WITH_JENNER"

dm_version = '20220513'
dm_file = f"MAP_VOCABULARY.RXNORM_{dm_version}.DRUG_MASTER_ACTIVE_AND_HISTORICAL"

asp_price = 'ASP_PRICE_2022JAN'
asp_xwalk = 'ASP_NDC_HCPCS_CROSSWALK_2022JAN'
asp_noc_price = 'ASP_NOC_PRICE_2022JAN'
asp_noc_xwalk = 'ASP_NOC_NDC_HCPCS_CROSSWALK_2022JAN'
nadac_xwalk = 'NADAC_20220322'

cpi_index = "CPI_INDEX_MEDICALSERVICES_20220627"
    
sample = 1
seed = 1234

In [25]:
# define output table names - easier to rerun intermediate steps
input_source = 'claim'
# input_source = 'encounter'

if input_source == 'encounter':
    prefix = f"RX_ENCOUNTERS_{rx_version}"
elif input_source == 'claim':
    prefix = 'RX_CERTIFIED_CLAIMS'
    
# get all claims, filter to PAID and cleaning
rx_enc_step1 = f"{prefix}_STEP1"

# quantity imputation
days_supply_quantity_dist = f"{prefix}_DAYS_SUPPLY_QUANTITY_DIST"
days_supply_quantity_guess = f"{prefix}_DAYS_SUPPLY_QUANTITY_GUESS"
quantity_guess = f"{prefix}_QUANTITY_GUESS"
rx_enc_step2 = f"{prefix}_STEP2"

# unit price imputation
asp_per_unit_price = f"{prefix}_ASP_PER_UNIT_PRICE"
nadac_per_unit_price = f"{prefix}_NADAC_PER_UNIT_PRICE"
median_unit_price = f"{prefix}_MEDIAN_UNIT_PRICE"
unit_price = f"{prefix}_UNIT_PRICE"

# final imputation
cost_per_days_supply = f"{prefix}_COST_PER_DAYS_SUPPLY"
rx_enc_impute = f"{prefix}_IMPUTED"

# define different variable names for different input tables
if input_source == 'encounter':
    input_table = rx_enc
    patient_key = 'upk_key2'
    claim_key = 'encounter_key'
    claim_date = 'claim_date'
    ndc_var = 'ndc'
    source_var = 'sources'
elif input_source == 'claim':
    input_table = rx_claim
    patient_key = 'upk_key2'
    claim_key = 'claim_id'
    claim_date = 'date_of_service'
    ndc_var = 'ndc11'
    source_var = 'source'




In [16]:
%%time
# query all Rx claims from encounters
# filter to Paid
# perform cleaning
sql = f"""
    CREATE OR REPLACE TABLE {rx_enc_step1} AS
    SELECT {patient_key}, {claim_key}, {claim_date}, rx.{ndc_var}, LEFT(rx.{ndc_var}, 9) AS ndc9, 
        days_supply, quantity_dispensed, 
        unit_of_measure, 
        payer_type_code, patient_state, {source_var},
        PHARMACY_SUBMITTED_COST, COPAY_COINSURANCE, PLAN_PAY_PRIMARY, PATIENT_PAY,
        PLAN_PAY_PRIMARY as plan_pay_actual, 
        case when 
            PLAN_PAY_PRIMARY is not null or PATIENT_PAY is not null then 
                COALESCE(PLAN_PAY_PRIMARY, 0) + COALESCE(PATIENT_PAY, 0)
            else null
        end as total_cost_actual,
        case when days_supply <= 365 and days_supply > 0 then days_supply else null end as days_supply_clean,
        percentile_cont(0.10) within group (order by quantity_dispensed) over (partition by rx.{ndc_var}) as quantity_10,
        percentile_cont(0.50) within group (order by quantity_dispensed) over (partition by rx.{ndc_var}) as quantity_50,
        percentile_cont(0.90) within group (order by quantity_dispensed) over (partition by rx.{ndc_var}) as quantity_90,
        case when quantity_dispensed > 0 and quantity_dispensed >= quantity_10/2 and quantity_dispensed <= quantity_90*2 and quantity_dispensed <= quantity_50*10 and quantity_dispensed <= 10000 then quantity_dispensed else null end as quantity_clean,
        percentile_cont(0.10) within group (order by plan_pay_actual) over (partition by rx.{ndc_var}) as plan_pay_10,
        percentile_cont(0.50) within group (order by plan_pay_actual) over (partition by rx.{ndc_var}) as plan_pay_50,
        percentile_cont(0.90) within group (order by plan_pay_actual) over (partition by rx.{ndc_var}) as plan_pay_90,
        case when plan_pay_actual = 0 or (plan_pay_actual > 0 and plan_pay_actual >= plan_pay_10/2 and plan_pay_actual <= plan_pay_90*2) then plan_pay_actual else null end as plan_pay_clean,
        percentile_cont(0.10) within group (order by total_cost_actual) over (partition by rx.{ndc_var}) as total_cost_10,
        percentile_cont(0.50) within group (order by total_cost_actual) over (partition by rx.{ndc_var}) as total_cost_50,
        percentile_cont(0.90) within group (order by total_cost_actual) over (partition by rx.{ndc_var}) as total_cost_90,
        case when total_cost_actual = 0 or (total_cost_actual > 0 and total_cost_actual >= total_cost_10/2 and total_cost_actual <= total_cost_90*2) then total_cost_actual else null end as total_cost_clean
    FROM {input_table} rx 
    -- SAMPLE ({sample}) SEED ({seed})
    WHERE transaction_type = 'PAID'
    ;
"""
execute_sql(sql)

CPU times: user 635 ms, sys: 60.3 ms, total: 696 ms
Wall time: 1h 43min 44s


In [18]:
# construct a frequency table as most likely days_supply - quantity combination for quantity imputation
# impute quantity by the top frequent days_supply - quantity pair per NDC, or just top frequent quantity

sql = f"""
    CREATE OR REPLACE TABLE {days_supply_quantity_dist} AS
    select {ndc_var}, days_supply_clean, quantity_clean, count(*) as n
    from {rx_enc_step1}
    where quantity_clean is not null and days_supply_clean is not null
    group by {ndc_var}, days_supply_clean, quantity_clean
    ;
"""
execute_sql(sql)
sql = f"""
    CREATE OR REPLACE TABLE {days_supply_quantity_guess} AS
    with add_row_number as (
        select *,
        row_number() over(partition by {ndc_var}, days_supply_clean order by n desc, quantity_clean desc) as row_number
        from {days_supply_quantity_dist}
    )
    select {ndc_var}, days_supply_clean, quantity_clean, n from add_row_number
    where row_number = 1
    ;
"""
execute_sql(sql)

sql = f"""
    CREATE OR REPLACE TABLE {quantity_guess} AS
    with ndc_quantity as (
        select {ndc_var}, quantity_clean, sum(n) as n
        from {days_supply_quantity_dist}
        group by {ndc_var}, quantity_clean
    ),
    add_row_number as (
        select *,
        row_number() over(partition by {ndc_var} order by n desc, quantity_clean desc) as row_number
        from ndc_quantity
    )
    select {ndc_var}, quantity_clean, n from add_row_number
    where row_number = 1
    ;
"""
execute_sql(sql)

In [ ]:
%%time
# impute and finalize quantity
sql = f"""
    CREATE OR REPLACE TABLE {rx_enc_step2} AS
    SELECT {patient_key}, {claim_key}, {claim_date}, e.{ndc_var}, ndc9, 
        quantity_dispensed,
        e.quantity_clean, 
        coalesce(dq.quantity_clean, q.quantity_clean) as quantity_imputed,
        coalesce(e.quantity_clean, dq.quantity_clean, q.quantity_clean) as quantity_final,
        case 
            when quantity_final is null and quantity_dispensed is not null then 'drop'
            when quantity_final is null then 'missing'
            when quantity_final = e.quantity_clean then 'orig'
            when quantity_final = dq.quantity_clean then 'imp_ds'
            when quantity_final = q.quantity_clean then 'imp'
            else 'err'
        end as quantity_source,
        days_supply, e.days_supply_clean,
        unit_of_measure, 
        payer_type_code, patient_state, {source_var},
        plan_pay_actual, plan_pay_clean, total_cost_actual, total_cost_clean
    FROM {rx_enc_step1} e
    LEFT JOIN {days_supply_quantity_guess} dq
    ON e.{ndc_var} = dq.{ndc_var} and e.days_supply_clean = dq.days_supply_clean
    LEFT JOIN {quantity_guess} q
    ON e.{ndc_var} = q.{ndc_var}
    ;
"""
execute_sql(sql)

In [23]:
# create asp per unit price
# HCPCS to NDC is not one to one mapping
# resolve duplicates in a super rough way...

sql = f"""
    CREATE OR REPLACE TABLE {asp_per_unit_price} AS
    SELECT DISTINCT 
        a1.hcpcs_code AS proc_code, a2.ndc2 AS ndc, 
        a1.payment_limit/IFNULL(IFNULL(TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(a1.hcpcs_code_dosage,'mg', ' mg'),',',''), ' ',0),3,2),TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(a2.hcpcs_dosage,'mg', ' mg'),',',''), ' ',0),3,2)), 1) AS asp_per_unit, 
        payment_limit, billunits, asp_per_unit*billunits AS asp_billunit_price
    FROM {asp_price} AS a1
    INNER JOIN {asp_xwalk} AS a2
    ON a1.hcpcs_code = a2.hcpcs_2022_code
    WHERE ndc IN (SELECT DISTINCT {ndc_var} FROM {input_table})
    UNION ALL 
    SELECT DISTINCT NULL AS proc_code, b2.ndc_or_alternate_id as ndc, 
    b1.payment_limit/IFNULL(IFNULL(TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(b1.dosage,'mg', ' mg'),',',''), ' ',0),3,2),TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(b2.dosage,'mg', ' mg'),',',''), ' ',0),3,2)), 1) AS asp_per_unit, 
    payment_limit, billunits, asp_per_unit*billunits AS asp_billunit_price
    FROM {asp_noc_price} AS b1
    INNER JOIN {asp_noc_xwalk} AS b2
    ON b1.drug_generic_name_trade_name_ = b2.drug_generic_name
    WHERE ndc IN (SELECT DISTINCT {ndc_var} FROM {input_table})
    ;
"""
execute_sql(sql)
sql = f"""
    CREATE OR REPLACE TABLE {asp_per_unit_price} AS
    SELECT *, 
        max(asp_per_unit) over (partition by ndc) as max_asp_per_unit
    from {asp_per_unit_price}
    qualify asp_per_unit = max_asp_per_unit
    ;
"""
execute_sql(sql)

In [24]:
# create nadac per unit price
## clean up effective date and build non-overlap intervals
sql = f"""
    CREATE OR REPLACE TABLE {nadac_per_unit_price} AS
    with nadac as (
    select distinct NDC, TO_DECIMAL(NADAC_PER_UNIT, 20, 9) as NADAC_PER_UNIT, NDC_DESCRIPTION, OTC, PRICING_UNIT, 
        TO_DATE(effective_date) as effective_date, CLASSIFICATION_FOR_RATE_SETTING
    from {nadac_xwalk}
    WHERE ndc IN (SELECT DISTINCT {ndc_var} FROM {input_table})
    )
    select ndc, nadac_per_unit, ndc_description, otc, pricing_unit, classification_for_rate_setting,
        '1970-01-01' as start_date, MIN(effective_date) OVER (PARTITION BY ndc) -1 as end_date
    from nadac
    qualify end_date = effective_date - 1
    union
    select ndc, nadac_per_unit, ndc_description, otc, pricing_unit, classification_for_rate_setting,
        effective_date as start_date, 
        coalesce(lead(effective_date) OVER (PARTITION BY ndc ORDER BY effective_date) -1, '2030-12-31') AS end_date
    from nadac
    order by ndc, start_date, end_date
    ;
"""
execute_sql(sql)

In [26]:
%%time
# calculate median unit price and merge with asp, nadac
# can run some more investigations
# change to ndc9, change to median_plan_pay
sql = f"""
    CREATE OR REPLACE TABLE {median_unit_price} AS
    with med_ndc9_year as (
    select ndc9, count(*) as n_claim_by_year, year({claim_date}) as year,
        -- median(total_cost_clean/quantity_final) as median_total_cost_per_unit, 
        median(plan_pay_clean/quantity_final) as median_plan_pay_per_unit_by_year
    from {rx_enc_step2}
    where quantity_final > 0 and plan_pay_clean > 0
        and left({ndc_var}, 5) != '00000' and left({ndc_var}, 5) != '99999'
    group by ndc9, year
    ),
    med_ndc9 as (
    select ndc9, count(*) as n_claim,
        -- median(total_cost_clean/quantity_final) as median_total_cost_per_unit, 
        median(plan_pay_clean/quantity_final) as median_plan_pay_per_unit
    from {rx_enc_step2}
    where quantity_final > 0 and plan_pay_clean > 0
        and left({ndc_var}, 5) != '00000' and left({ndc_var}, 5) != '99999'
    group by ndc9
    ),
    ndc_to_ndc9 as (
    select distinct {ndc_var} as ndc, ndc9
    from {rx_enc_step2}
    where quantity_final is not null and plan_pay_clean is not null
    ),
    by_year as (
    select my.ndc9, ndc, year, n_claim, n_claim_by_year,
        median_plan_pay_per_unit, median_plan_pay_per_unit_by_year,
        DATE_FROM_PARTS(year, 1, 1) as year_start_date,
        case
            when n_claim_by_year >= 100 then median_plan_pay_per_unit_by_year
            else median_plan_pay_per_unit
        end as median_per_unit
    from med_ndc9_year my
    left join med_ndc9 m
        on my.ndc9 = m.ndc9
    left join ndc_to_ndc9 x
        on my.ndc9 = x.ndc9
    )
    -- this is to fill out years when there is no such ndc.
    select 
        ndc9, ndc, median_per_unit,
        '1970-01-01' as start_date, 
        min(year_start_date) OVER (PARTITION BY ndc) -1 as end_date,
        n_claim, n_claim_by_year, year_start_date,
        median_plan_pay_per_unit, median_plan_pay_per_unit_by_year
    from by_year
    qualify year_start_date-1 = end_date
    union all
    select 
        ndc9, ndc, median_per_unit,
        year_start_date as start_date, 
        lead(year_start_date) OVER (PARTITION BY ndc ORDER BY year_start_date) -1 as end_date,
        n_claim, n_claim_by_year, year_start_date,
        median_plan_pay_per_unit, median_plan_pay_per_unit_by_year
    from by_year
    qualify end_date is not null
    union all
    select 
        ndc9, ndc, median_per_unit,
        max(year_start_date) OVER (PARTITION BY ndc) as start_date,
        '2030-12-31' as end_date, 
        n_claim, n_claim_by_year, year_start_date,
        median_plan_pay_per_unit, median_plan_pay_per_unit_by_year
    from by_year
    qualify year_start_date = start_date
    ;
"""
execute_sql(sql)

CPU times: user 305 ms, sys: 0 ns, total: 305 ms
Wall time: 12min 49s


In [27]:
%%time
# merge unit price together
# assume tables include unit price from 1970 - 2030 (those are arbitrary early/late dates)

sql = f"""
    CREATE OR REPLACE TABLE {unit_price} AS
    with median_nadac as (
    select 
        coalesce(med.ndc, nadac.ndc) as ndc, 
        n_claim,
        coalesce(greatest(med.start_date, nadac.start_date),med.start_date, nadac.start_date) as tmp_start_date,
        coalesce(least(med.end_date, nadac.end_date),med.end_date, nadac.end_date) as tmp_end_date,
        median_per_unit, nadac_per_unit, median_plan_pay_per_unit, median_plan_pay_per_unit_by_year
    from {median_unit_price} med
    full join {nadac_per_unit_price} nadac
    on med.ndc = nadac.ndc 
        and greatest(med.start_date, nadac.start_date) <= least(med.end_date, nadac.end_date) 
    )
    select distinct 
        coalesce(median_nadac.ndc, asp.ndc) as ndc,
        n_claim, 
        coalesce(tmp_start_date, '1970-01-01') as start_date,
        coalesce(tmp_end_date, '2030-12-31') as end_date,
        coalesce(median_per_unit, nadac_per_unit, asp_per_unit) as unit_price_final,
        case 
            when abs(unit_price_final - median_plan_pay_per_unit_by_year) < 0.00001 then 'median_by_year'
            when abs(unit_price_final - median_per_unit) < 0.00001 then 'median'
            when abs(unit_price_final - nadac_per_unit) < 0.00001 then 'nadac'
            when abs(unit_price_final - asp_per_unit) < 0.00001 then 'asp'
            else 'err'
        end as unit_price_source,
        greatest(least(case 
            when unit_price_source = 'asp' or unit_price_source = 'nadac' then 2022
            when start_date = '1970-01-01' then year(end_date + 1) 
            when end_date = '2030-12-31' then year(start_date)
            else year(start_date)
        end, 2022), 2012) as price_year,
        median_per_unit, nadac_per_unit, asp_per_unit, median_plan_pay_per_unit, median_plan_pay_per_unit_by_year
    from median_nadac
    full join {asp_per_unit_price} asp
    on median_nadac.ndc = asp.ndc
    ;
"""
execute_sql(sql)

CPU times: user 6.03 ms, sys: 0 ns, total: 6.03 ms
Wall time: 4.92 s


In [28]:
%%time
# summarize cost by ndc, days_supply
# as a backup plan for ndcs with missing quantity/ days supply fill rate is much better
sql = f"""
    CREATE OR REPLACE TABLE {cost_per_days_supply} AS
    select {ndc_var}, days_supply_clean, median(plan_pay_clean) as cost_ds_impute
    from {rx_enc_step1}
    where days_supply_clean > 0 and plan_pay_clean > 0
    group by {ndc_var}, days_supply_clean
    ;
"""
execute_sql(sql)

CPU times: user 41.1 ms, sys: 0 ns, total: 41.1 ms
Wall time: 5min 25s


In [ ]:
%%time
# impute final cost
# merge with drug master table for future improvement

sql = f"""
    CREATE OR REPLACE TABLE {rx_enc_impute} AS
    with dm as (
        select distinct ndc, start_date, dfg_name_array,
            max(start_date) over (partition by ndc) as max_start_date
        from {dm_file}
        qualify start_date = max_start_date
    )
    select {patient_key}, {claim_key}, {claim_date}, e.{ndc_var}, ndc9, 
        plan_pay_actual, plan_pay_clean, total_cost_actual, total_cost_clean, 
        quantity_final * unit_price_final as cost_impute,
        price_year, 
        cost_impute * cpi2.cpi_ratio / cpi1.cpi_ratio as cost_impute_adj,
        case 
            when plan_pay_clean is null then cost_impute_adj
            when cost_impute_adj is null then plan_pay_clean
            when plan_pay_clean = 0 then plan_pay_clean
            when cost_impute_adj > 0 and plan_pay_clean > 0 and cost_impute_adj/plan_pay_clean > 10 or plan_pay_clean/cost_impute_adj > 10 then least(cost_impute_adj, plan_pay_clean)
            else plan_pay_clean
        end as cost_final,
        -- coalesce(plan_pay_clean, cost_impute_adj) as cost_final,
        cost_impute_adj as cost_final_sentinel,
        round(cost_final, 2) as cost_final_round,
        round(cost_final_sentinel, 2) as cost_final_sentinel_round,
        case 
            when cost_final is null then null
            when abs(cost_final - plan_pay_clean) < 0.00001 then 'orig'
            when abs(cost_final - cost_impute_adj) < 0.00001 then 'imp'
            else 'err'
        end as cost_final_source,
        cost_ds_impute,
        cpi2.cpi_ratio as price_year_cpi_ratio, cpi1.cpi_ratio as claim_year_cpi_ratio,
        quantity_final, quantity_source, unit_price_final, unit_price_source,
        median_per_unit, nadac_per_unit, asp_per_unit, 
        unit_of_measure, 
        payer_type_code, patient_state, {source_var}, 
        dm.dfg_name_array
    from {rx_enc_step2} e
    left join {unit_price} up
    on e.{ndc_var} = up.ndc and e.{claim_date} between up.start_date and up.end_date
    left join {cpi_index} cpi1
    on year({claim_date}) = cpi1.year
    left join {cpi_index} cpi2
    on price_year = cpi2.year
    left join dm
    on e.{ndc_var} = dm.ndc
    left join {cost_per_days_supply} cds
    on e.{ndc_var} = cds.{ndc_var} and e.days_supply_clean = cds.days_supply_clean
    ;
"""
execute_sql(sql)

In [ ]:
# Testing
sql = f"""
    CREATE OR REPLACE TABLE test AS
    select ndc, start_date, end_date, count(*) as n
    from {unit_price} e
    group by ndc, start_date, end_date
    having n > 1
    ;
"""

# sql = f"""
#     CREATE OR REPLACE TABLE test AS
#     select *
#     from {unit_price} e
#     where ndc = '00002751001'
#     ;
# """

# sql = f"""
#     CREATE OR REPLACE TABLE test AS
#     select *
#     from {asp_per_unit_price} e
#     where ndc = '00002751001'
#     ;
# """

sql = f"""
    CREATE OR REPLACE TABLE test AS
    select ndc, count(*) as n
    from (
    
    select distinct ndc, start_date, dfg_name_array,
        max(start_date) over (partition by ndc) as max_start_date
    from {dm_file}
    qualify start_date = max_start_date
    )
    group by ndc
    having n > 1
    ;
"""
sql = f"""
    CREATE OR REPLACE TABLE test AS
    select *
    from {dm_file}
    where ndc = '35356001703'
    ;
"""
# only one code failed the duplicate test, let it go
execute_sql(sql)

In [ ]:
#ORIGINAL CODE, Don't run the cell

----------
-- TABLE: "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST"
-- GOAL: Create Payer Paid Amount (Cost) Table for Rx Encounters using imputed amounts and publicly-available price files (NADAC and ASP)
----------
-- APPROACH:
-- 0a. Impute NADAC price = NADAC_per_unit
-- 0b. Impute ASP price = ASP_per_unit
-- 1. Quantity_dispensed = quantity_actual
-- 2. Clean outliers: > 95th percentile --> 95th percentile, < 5th percentile --> 5th percentile OVER ndc, DS = quantity_clean
-- 3. Impute NULLs: MEDIAN (quantity_clean) OVER NDC, DS = quantity_impute
-- 4. COALESCE (quantity_clean, quantity_impute) = quantity_final
-- 5. Plan_pay_primary = plan_pay_actual
-- 6. Clean outliers:  95th percentile --> 95th percentile, < 5th percentile --> 5th percentile OVER ndc, quantity_final = plan_pay_clean
-- 7. Impute NULLS: MEDIAN (plan_pay_clean/quantity_final) = plan_pay_impute_per_unit
-- 8. COALESCE (plan_pay_actual, plan_pay_impute_per_unit*quantity_final, NADAC_per_unit*quantity_final, ASP_per_unit*quantity_final)

CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS
SELECT encounter_key, claim_date, ndc, days_supply
, quantity_dispensed AS quantity_actual
, plan_pay_primary AS plan_pay_actual
, LEFT(ndc,9) AS ndc_left9 -- for partitions since first 9 digits remove package size
FROM "MAP_ENCOUNTERS"."RX_ENCOUNTERS_20220511"."RX_ENCOUNTER_LS_GA"
WHERE transaction_type = 'PAID'

; 

-- Step 0
CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS
WITH asp AS 
(SELECT DISTINCT a1.hcpcs_code AS proc_code, a2.ndc2 AS ndc_code, 
    a1.payment_limit/IFNULL(IFNULL(TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(a1.hcpcs_code_dosage,'mg', ' mg'),',',''), ' ',0),3,2),TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(a2.hcpcs_dosage,'mg', ' mg'),',',''), ' ',0),3,2)), 1) AS asp_per_unit, payment_limit, billunits, asp_per_unit*billunits AS asp_billunit_price
    FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."ASP_PRICE_2022JAN" AS a1
    INNER JOIN "SANDBOX_KOMODO"."PROJECT_CURRENCY"."ASP_NDC_HCPCS_CROSSWALK_2022JAN" AS a2
    ON a1.hcpcs_code = a2.hcpcs_2022_code
    WHERE ndc_code IN (SELECT DISTINCT ndc FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST")
    UNION ALL SELECT DISTINCT NULL AS proc_code, b2.ndc_or_alternate_id, 
    b1.payment_limit/IFNULL(IFNULL(TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(b1.dosage,'mg', ' mg'),',',''), ' ',0),3,2),TRY_TO_DECIMAL(SPLIT_PART(REPLACE(REPLACE(b2.dosage,'mg', ' mg'),',',''), ' ',0),3,2)), 1) AS asp_per_unit, payment_limit, billunits, asp_per_unit*billunits AS asp_billunit_price
    FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."ASP_NOC_PRICE_2022JAN" AS b1
    INNER JOIN "SANDBOX_KOMODO"."PROJECT_CURRENCY"."ASP_NOC_NDC_HCPCS_CROSSWALK_2022JAN" AS b2
    ON b1.drug_generic_name_trade_name_ = b2.drug_generic_name
    WHERE ndc_or_alternate_id IN (SELECT DISTINCT ndc FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST")
)

, nadac AS 
(
-- 1:many prices per nadac is driven by multiple effective_dates in the file
-- per CMS, these dates are designed to not overlap & each NADAC file is a full replacement of previous files: https://www.medicaid.gov/medicaid-chip-program-information/by-topics/prescription-drugs/ful-nadac-downloads/nadacmethodology.pdf
-- creating an "effective window" where available with lead function ; if Rx claim occurs during an effective_window, will use that NADAC price
-- for any claims that were filled before the first effective date of that drug, will use the price as of the earliest effective date for that drug
SELECT *
, MIN(TO_DATE(effective_date)) OVER (PARTITION BY ndc) AS first_effective_date
, LEAD(TO_DATE(effective_date)) OVER (PARTITION BY ndc ORDER BY TO_DATE(effective_date)) AS next_effective_date
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."NADAC_20220322"
)
 
SELECT 
encounter_key, claim_date, t1.ndc, ndc_left9, days_supply
, quantity_actual
, plan_pay_actual
, nadac_per_unit
, asp_per_unit
FROM  "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS t1
LEFT JOIN nadac AS t2
ON t1.ndc = t2.ndc 
LEFT JOIN asp AS t3
ON t1.ndc = t3.ndc_code
WHERE CASE WHEN claim_date BETWEEN effective_date AND next_effective_date THEN claim_date BETWEEN effective_date AND next_effective_date -- if available, will select the price during the "effective window"
WHEN claim_date < first_effective_date THEN claim_date < first_effective_date AND effective_date = first_effective_date  -- if claim was before the first NDC's first effective_date, will use the price as of the first 
ELSE effective_date IS NULL END -- in case there is no NADAC price/effective_date
; 

-- previous query increases row count by 5x due to nadac join, so need this distinct step to reduce count and optimize subsequent queries
CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS
SELECT DISTINCT * 
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" 

;

CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST_OUTLIERS" AS
-- Step 2
--WITH outlier_quantities AS 
(SELECT encounter_key
, claim_date
, PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY quantity_actual) OVER (PARTITION BY ndc_left9, days_supply) AS quantity_99th
, PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY quantity_actual) OVER (PARTITION BY ndc_left9, days_supply) AS quantity_95th
 , PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY quantity_actual) OVER (PARTITION BY ndc_left9, days_supply) AS quantity_75th
, PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY quantity_actual) OVER (PARTITION BY ndc_left9, days_supply) AS quantity_25th
, PERCENTILE_CONT(0.05) WITHIN GROUP (ORDER BY quantity_actual) OVER (PARTITION BY ndc_left9, days_supply) AS quantity_5th 
, PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY quantity_actual) OVER (PARTITION BY ndc_left9, days_supply) AS quantity_1st 
, CASE WHEN quantity_actual > quantity_95th THEN quantity_95th
 WHEN quantity_actual < quantity_5th THEN quantity_5th
 ELSE quantity_actual END AS quantity_clean
 , CASE WHEN quantity_actual > quantity_99th THEN quantity_99th
 WHEN quantity_actual < quantity_5th THEN quantity_5th
 ELSE quantity_actual END AS quantity_clean_5th_99th
 , CASE WHEN quantity_actual > quantity_99th THEN quantity_99th
 WHEN quantity_actual < quantity_1st THEN quantity_1st
 ELSE quantity_actual END AS quantity_clean_1st_99th
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST"
)

;

-- Step 3-4 
CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS 
SELECT t1.*
, quantity_clean
, quantity_clean_5th_99th
, quantity_clean_1st_99th
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS t1
LEFT JOIN "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST_OUTLIERS" AS t2
ON t1.encounter_key = t2.encounter_key
AND t1.claim_date = t2.claim_date
; 

CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS 
SELECT DISTINCT *
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" 
; 

-- join + window functions = long compute time ; have to split in separate steps
-- multiple median window functions in 1 query also = long compute time ; splitting in separate steps
-- calc medians only with groupby & join back to paid table
CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST_MEDIANS" AS 
SELECT ndc_left9, days_supply
, MEDIAN(quantity_clean) AS  quantity_impute -- used median in Rx because quantity billing is relatively  consistent in Rx claims
, MEDIAN(quantity_clean_5th_99th) AS  quantity_impute_5th_99th 
, MEDIAN(quantity_clean_1st_99th) AS quantity_impute_1st_99th 
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS t1
GROUP BY 1, 2
; 

CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS 
SELECT t1.*
, COALESCE(quantity_clean, quantity_impute) AS quantity_final
, COALESCE(quantity_clean_5th_99th, quantity_impute_5th_99th) AS quantity_final_5th_99th
, COALESCE(quantity_clean_1st_99th, quantity_impute_1st_99th) AS quantity_final_1st_99th
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS t1
LEFT JOIN "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST_MEDIANS" AS t2
ON t1.ndc_left9 = t2.ndc_left9
AND t1.days_supply = t2.days_supply
;

CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS
-- Step 6
WITH remove_zero_plan_pay_for_imputation AS 
(
SELECT encounter_key, claim_date, ndc_left9, quantity_final, plan_pay_actual
, quantity_final_5th_99th, quantity_final_1st_99th
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST"  
WHERE plan_pay_actual > 0
)

, outlier_plan_pay AS 
(SELECT encounter_key
, claim_date
, PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final) AS plan_pay_95th
, PERCENTILE_CONT(0.05) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final) AS plan_pay_5th 
, CASE WHEN plan_pay_actual > plan_pay_95th THEN plan_pay_95th
 WHEN plan_pay_actual < plan_pay_5th THEN plan_pay_5th
 ELSE plan_pay_actual END AS plan_pay_clean
, PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final) AS plan_pay_99th
, PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final) AS plan_pay_1st 
, CASE WHEN plan_pay_actual > plan_pay_99th THEN plan_pay_99th
 WHEN plan_pay_actual < plan_pay_1st THEN plan_pay_1st
 ELSE plan_pay_actual END AS plan_pay_clean_1st_99th
, PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final_5th_99th) AS plan_pay_99th_qty5th_99th
, PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final_5th_99th) AS plan_pay_1st_qty5th_99th
, CASE WHEN plan_pay_actual > plan_pay_99th_qty5th_99th THEN plan_pay_99th_qty5th_99th
 WHEN plan_pay_actual < plan_pay_1st_qty5th_99th THEN plan_pay_1st_qty5th_99th
 ELSE plan_pay_actual END AS plan_pay_clean_1st_99th_qty5th_99th
, PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final_1st_99th) AS plan_pay_99th_qty1st_99th
, PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY plan_pay_actual) OVER (PARTITION BY ndc_left9, quantity_final_1st_99th) AS plan_pay_1st_qty1st_99th
, CASE WHEN plan_pay_actual > plan_pay_99th_qty1st_99th THEN plan_pay_99th_qty1st_99th
 WHEN plan_pay_actual < plan_pay_1st_qty1st_99th THEN plan_pay_1st_qty1st_99th
 ELSE plan_pay_actual END AS plan_pay_clean_1st_99th_qty1st_99th
FROM remove_zero_plan_pay_for_imputation
)

-- Step 7-8
SELECT t1.*
, plan_pay_clean
, MEDIAN(plan_pay_clean/IFF(quantity_final=0, 0.5, quantity_final)) OVER (PARTITION BY ndc_left9) AS  plan_pay_impute_per_unit -- some spot checking of "zero" quantity_final values were vaccines (which have quantities of 0.5 ml) -- this isn't perfect, but balances maintaining integrity of "zero" values in the actuals and preventing "division by zero" errors
, COALESCE(plan_pay_clean, plan_pay_impute_per_unit*quantity_final, nadac_per_unit*quantity_final, asp_per_unit*quantity_final) AS plan_pay_final
, COALESCE(plan_pay_impute_per_unit*quantity_final, nadac_per_unit*quantity_final, asp_per_unit*quantity_final) AS plan_pay_fullyimputed_sentinel
, IFF (plan_pay_impute_per_unit*quantity_final>0, plan_pay_impute_per_unit*quantity_final, COALESCE(nadac_per_unit*quantity_final, asp_per_unit*quantity_final)) AS plan_pay_skipzeros
, plan_pay_clean_1st_99th
, MEDIAN(plan_pay_clean_1st_99th/IFF(quantity_final=0, 0.5, quantity_final)) OVER (PARTITION BY ndc_left9) AS  plan_pay_impute_per_unit_1st_99th -- some spot checking of "zero" quantity_final values were vaccines (which have quantities of 0.5 ml) -- this isn't perfect, but balances maintaining integrity of "zero" values in the actuals and preventing "division by zero" errors
, COALESCE(plan_pay_clean_1st_99th, plan_pay_impute_per_unit_1st_99th*quantity_final, nadac_per_unit*quantity_final, asp_per_unit*quantity_final) AS plan_pay_final_1st_99th
, COALESCE(plan_pay_impute_per_unit_1st_99th*quantity_final, nadac_per_unit*quantity_final, asp_per_unit*quantity_final) AS plan_pay_fullyimputed_sentinel_1st_99th
, plan_pay_clean_1st_99th_qty5th_99th
, MEDIAN(plan_pay_clean_1st_99th_qty5th_99th/IFF(quantity_final_5th_99th=0, 0.5, quantity_final_5th_99th)) OVER (PARTITION BY ndc_left9) AS  plan_pay_impute_per_unit_1st_99th_qty5th_99th -- some spot checking of "zero" quantity_final values were vaccines (which have quantities of 0.5 ml) -- this isn't perfect, but balances maintaining integrity of "zero" values in the actuals and preventing "division by zero" errors
, COALESCE(plan_pay_clean_1st_99th_qty5th_99th, plan_pay_impute_per_unit_1st_99th_qty5th_99th*quantity_final_5th_99th, nadac_per_unit*quantity_final_5th_99th, asp_per_unit*quantity_final_5th_99th) AS plan_pay_final_1st_99th_qty5th_99th
, COALESCE(plan_pay_impute_per_unit_1st_99th_qty5th_99th*quantity_final_5th_99th, nadac_per_unit*quantity_final_5th_99th, asp_per_unit*quantity_final_5th_99th) AS plan_pay_fullyimputed_sentinel_1st_99th_qty5th_99th
, plan_pay_clean_1st_99th_qty1st_99th
, MEDIAN(plan_pay_clean_1st_99th_qty1st_99th/IFF(quantity_final_1st_99th=0, 0.5, quantity_final_1st_99th)) OVER (PARTITION BY ndc_left9) AS  plan_pay_impute_per_unit_1st_99th_qty1st_99th -- some spot checking of "zero" quantity_final values were vaccines (which have quantities of 0.5 ml) -- this isn't perfect, but balances maintaining integrity of "zero" values in the actuals and preventing "division by zero" errors
, COALESCE(plan_pay_clean_1st_99th_qty1st_99th, plan_pay_impute_per_unit_1st_99th_qty1st_99th*quantity_final_1st_99th, nadac_per_unit*quantity_final_1st_99th, asp_per_unit*quantity_final_1st_99th) AS plan_pay_final_1st_99th_qty1st_99th
, COALESCE(plan_pay_impute_per_unit_1st_99th_qty1st_99th*quantity_final_1st_99th, nadac_per_unit*quantity_final_1st_99th, asp_per_unit*quantity_final_1st_99th) AS plan_pay_fullyimputed_sentinel_1st_99th_qty1st_99th
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS t1
LEFT JOIN outlier_plan_pay AS t2
ON t1.encounter_key = t2.encounter_key

;

CREATE OR REPLACE TABLE "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST" AS
SELECT DISTINCT *
FROM "SANDBOX_KOMODO"."PROJECT_CURRENCY"."RX_ENCOUNTERS_20220511_PAID_COST"
